In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

In [83]:
df = pd.read_excel('data.xlsx', header=None)
df

,0,1,2,3,4,5,6,7
0,11,5,4,7,16,6,5,7
1,13,6,5,7,12,5,4,6
2,9,5,5,11,29,21,17,20
3,27,13,9,10,16,6,5,7
4,11,5,5,6,12,7,7,10
5,15,10,9,11,15,10,10,16
6,26,21,23,36,50,45,45,49
7,57,43,40,44,52,43,42,45
8,52,41,39,41,48,35,34,35
9,42,34,36,43,55,48,54,65


In [84]:
data = df.values
requirements = data.flatten()
requirements.shape

(104,)

## 问题一 回归模型

具体条件：

设 $H$ 为已有的熟练工数， $h$ 为新手数（等于本周买入的数量）， $H_\text{r}$ 为保养中的熟练工， $H_\text{w}$ 为工作中的熟练工， $H_\text{t}$ 为教学中的熟练工，用上标 $h^{(t)}$ 表示第 $t$ 周的新手数，则有

In [85]:
requirements1 = requirements[0:8]
k = 10
requirements1

array([11,  5,  4,  7, 16,  6,  5,  7], dtype=int64)

In [86]:
hand_new = cp.Variable(8, integer=True)
hand_work = cp.Variable(8, integer=True)
hand_rest = cp.Variable(8, integer=True)
hand_teach = cp.Variable(8, integer=True)
hand_skill = cp.Variable(8, integer=True)
body_new = cp.Variable(8, integer=True)
body_work = cp.Variable(8, integer=True)
body_rest = cp.Variable(8, integer=True)
body_tested = cp.Variable(8, integer=True)

增加一周内各个数量的制约条件

In [87]:
cons = [
    hand_work == 4 * requirements1,
    body_work == requirements1,
    k * hand_teach >= hand_new,
    hand_skill == hand_work + hand_rest + hand_teach,
    body_tested == body_work + body_rest,
    hand_new >= 0,
    hand_rest >= 0,
    hand_work >= 0,
    hand_teach >= 0,
    body_new >= 0,
    body_rest >= 0,
    body_work >= 0
]

增加一周之间的制约条件

In [88]:
for t in range(7):
    cons.append(hand_rest[t+1] >= hand_work[t]),
    cons.append(hand_skill[t+1] == hand_skill[t] + hand_new[t])
    cons.append(body_tested[t+1] == body_tested[t] + body_new[t])

初始值的制约条件

In [89]:
cons += [
    hand_skill[0] == 50,
    body_tested[0] == 13
]

In [90]:
obj = cp.Minimize(
    200 * cp.sum(body_new) + 
    100 * cp.sum(hand_new) + 
    5 * cp.sum(hand_rest) + 
    10 * cp.sum(body_rest) +
    10 * cp.sum(hand_new + hand_teach)
)

In [91]:
prob = cp.Problem(obj, cons)
prob.solve()

7625.0

In [92]:
hand_work.value, body_work.value, prob.value

(array([44., 20., 16., 28., 64., 24., 20., 28.]),
 array([11.,  5.,  4.,  7., 16.,  6.,  5.,  7.]),
 7625.0)

In [95]:
hand_new.value, hand_teach.value, body_new.value

(array([14.,  0.,  0., 28.,  0.,  0.,  0.,  0.]),
 array([2., 0., 0., 3., 0., 0., 0., 0.]),
 array([0., 0., 0., 3., 0., 0., 0., 0.]))